# Benchmarks and Tests of Maps for Single-Element Cluster Substrates 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
default_fontsize = plt.rcParams['font.size']
publication_fontsize_large = 20
publication = True
if publication: plt.rcParams.update({'font.size': publication_fontsize_large})

## Pt Nanocluster

Generate the ASE.Atoms instance using the fcc111 build function. The atom at the center of the cell on the top face is index number 42.

In [ ]:
from ase.cluster.cubic import FaceCenteredCubic
surfaces = [(1, 0, 0), (1, 1, 0), (1, 1, 1)]
layers = [5, 8, 5]
lc = 3.94000
PtCluster = FaceCenteredCubic('Pt', surfaces, layers, latticeconstant=lc)
PtCluster.cell = 30 * np.eye(3)
PtCluster.positions += [15, 15, 15]

In [ ]:
from ase.visualize import view
view(PtCluster, viewer='x3d')

In [ ]:
# uncomment to save the cluster coordinates to a file
# PtCluster.write('PtCluster.xyz')

From the `ASE.Cell` and the generated `ASE.Atoms`, create a `MapSy.Grid` and a `MapSy.System`

In [ ]:
from mapsy.data import Grid
grid: Grid = Grid(cell=PtCluster.cell)

In [ ]:
from mapsy.data import System
system: System = System(grid, PtCluster, dimension=0, axis=1)

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "ionic", "cutoff": 40, "alpha": 1.2, "spread" : 0.8, "threshold": 0.5})
contactspace = ContactSpaceGenerator(contactspacesettings).generate(system)

In [ ]:
contactspace.data['probability'].describe()

In [ ]:
contactspace.data

In [ ]:
contactspace.nregions

In [ ]:
contactspace.boundary.gradient.modulus.plotprojections([15,15,15])

In [ ]:
from mapsy.symfunc.input import SymmetryFunctionsModel, SymFuncModel
from mapsy.symfunc.parser import SymmetryFunctionsParser
symfuncsettings = SymmetryFunctionsModel.parse_obj({"functions": [SymFuncModel.parse_obj({"type":"ac","radius":4.5,"order":10,"compositional":False,"structural":True}),SymFuncModel.parse_obj({"type":"ac","radius":4.5,"order":10,"compositional":False,"structural":True,"radial":False})]})
symmetryfunctions = SymmetryFunctionsParser(symfuncsettings).parse()

In [ ]:
from mapsy.maps import Maps
maps = Maps(system,symmetryfunctions,contactspace)

In [ ]:
#data = maps.atcontactspace()
#data.to_csv('maps.csv')
# To save time, we load the data from a file
maps.data = pd.read_csv('maps.csv', index_col=0)
maps.features = maps.data.columns.drop(['x','y','z'])

In [ ]:
maps.data

We can visualize features to check how they look using `Maps.plot(feature: str)` or `Maps.plot(index: int)`. NOTE: to get the top face of the slab, we need to select `region=1`. 

In [ ]:
fig, axes = maps.scatter(index=0, cmap='Spectral', set_aspect='scaled')
axes.set_xlabel('x (Å)')
axes.set_ylabel('y (Å)')
axes.set_title('')
plt.show()

## Dimensionality Reduction (PCA)

For visualization and post-processing purposes, perform dimensionality reduction on the generated features. Three components are useful for 2D and 3D plots.

In [ ]:
fig, ax1, ax2 = maps.reduce(scale=True)
if (publication) : 
    ax1.set_title('PCA')
    fig.tight_layout()

In [ ]:
npca = 5
maps.reduce(npca, scale=True)

We can visually inspect how the PCAs correlate with the Cartesian coordinates of the points (e.g., PC0 still distinguishing between atop positions, while PC1 correlated with the distance from the defect).

In [ ]:
fig, gs = maps.scatter_pca_grid(index=0,cmap='Spectral',set_aspect='equal',s=70, alpha=0.05)
fig.tight_layout()

We can also verify how the contact space is transformed (folded) in the symmetry function space. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4*1))
fig.subplots_adjust(hspace=0.3)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_title("")
x1m = maps.data['pca0'].values.astype(np.float64)
x2m = maps.data['pca1'].values.astype(np.float64)
fm = maps.data[maps.features[0]].values.astype(np.float64)
fmin = np.min(fm)
fmax = np.max(fm)
scatter = ax.scatter(x1m,x2m,c=fm,vmin=fmin,vmax=fmax,cmap='Spectral',alpha=0.2,s=60,edgecolors='black')
ax.axis('on')
plt.show()

## Perform Clustering on Generated Features

Use SpectralClustering to find N clusters in the featured data. 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import SpectralClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
import scipy.spatial.distance as distance
from itertools import combinations

def cluster(self, nclusters=None, features=None, maxclusters=20, ntries = 1, random_state=None, scale=False):
    """ 
    
    """
    # Select the features for clustering
    if features is None:
        self.cluster_features = self.features
    else:
        self.cluster_features = features
    X = self.data[self.cluster_features].values.astype(np.float64)
    if scale : 
        X = StandardScaler().fit_transform(X)
    if nclusters is not None:
        if random_state is None:
            # If we performed a screening, use the best random state
            if self.best_clusters is not None:
                if nclusters in self.best_clusters['nclusters'].values:
                    self.random_state = self.best_clusters[self.best_clusters['nclusters']==nclusters]['random_state'].values[0]
                    print(f'Use best random state = {self.random_state} from screening')
                else:
                    self.random_state = np.random.randint(0,1000)
                    print(f'Use new random state = {self.random_state}')
            # Otherwise, pick a random number
            else:
                self.random_state = np.random.randint(0,1000)
                print(f'Use new random state = {self.random_state}')
        else:
            self.random_state = random_state
            print(f'Use given random state = {self.random_state}')
#        labels = SpectralClustering(n_clusters = nclusters, random_state=self.random_state).fit_predict(X)
        labels = GaussianMixture(n_components=nclusters).fit_predict(X)
        self.data['Cluster'] = labels
        # Store number of clusters
        self.nclusters = nclusters
        # Compute the cluster centers in features space
        self.cluster_centers = self.data.groupby('Cluster')[self.cluster_features].mean().values.astype(np.float64)
        # Compute the number of points in each cluster
        self.cluster_sizes = self.data.groupby('Cluster').size().values
        # Generate clusters connectivity matrix
        self.cluster_graph = self.graph(self.data['Cluster'])
        self.cluster_edges = self.cluster_graph.copy()
        for i in range(nclusters):
            self.cluster_edges[i,i] = 0
    else:
        cluster_range = range(2, maxclusters)
        cluster_random_states = []
        cluster_sizes = []
        silhouette_scores = []
        db_indexes = []
        # Loop over different numbers of clusters
        for nclusters in cluster_range:
            random_states = np.random.randint(0,1000,ntries)
            for random_state in random_states:
                cluster_random_states.append(random_state)
#                labels = SpectralClustering(n_clusters=nclusters, random_state=random_state).fit_predict(X)
                labels = GaussianMixture(n_components=nclusters, random_state=random_state).fit_predict(X)
                actual_nclusters = len(np.unique(labels))
                cluster_sizes.append(actual_nclusters)
                score = silhouette_score(X, labels)
                silhouette_scores.append(score)
                index = davies_bouldin_score(X, labels)
                db_indexes.append(index)
        # Store all clusters data
        self.cluster_screening = pd.DataFrame({'nclusters': cluster_sizes, 'random_state': cluster_random_states, 'silhouette_score': silhouette_scores, 'db_index': db_indexes})
        # Find the best clusters according to Silhouette Score
        best_db = self.cluster_screening.loc[self.cluster_screening.groupby('nclusters')['db_index'].idxmin()]
        best_sil = self.cluster_screening.loc[self.cluster_screening.groupby('nclusters')['silhouette_score'].idxmax()]
        self.best_clusters = best_db 
        # Plot Silhouette Scores
        fig, ax1 = plt.subplots()
        # Plot Silhouette Scores on the left y-axis
        ax1.scatter(cluster_sizes, silhouette_scores, color='b', marker='o', label='Silhouette Score')
        ax1.plot(best_db['nclusters'],best_db['silhouette_score'], '-', color='b')
        ax1.plot(best_sil['nclusters'],best_sil['silhouette_score'], ':', color='b')
        ax1.set_xlabel('Number of Clusters')
        ax1.set_ylabel('Silhouette Score', color='b')
        ax1.tick_params(axis='y', labelcolor='b')
        # Create a second y-axis to the right for Davies-Bouldin Index
        ax2 = ax1.twinx()
        ax2.scatter(cluster_sizes, db_indexes, color='r', marker='s', label='Davies-Bouldin Index')
        ax2.plot(best_db['nclusters'],best_db['db_index'], '-', color='r')
        ax2.plot(best_sil['nclusters'],best_sil['db_index'], ':', color='r')
        ax2.set_ylabel('Davies-Bouldin Index', color='r')
        ax2.tick_params(axis='y', labelcolor='r')
        # Title and grid
        ax1.set_title('Silhouette Score and Davies-Bouldin Index vs. Number of Clusters')
        ax1.grid(True)
        return fig, ax1, ax2

In [ ]:
ntries = 1
if publication: ntries = 3
fig, ax1, ax2 = cluster(maps,maxclusters=30, ntries=ntries)
if publication: 
    ax1.set_title('')
    ax2.set_title('')
plt.show()

In [ ]:
cluster(maps,nclusters=16)

Given the clusters, plot the connectivity matrix

In [ ]:
plt.matshow(maps.cluster_edges)

Given the clusters and the connectivity, find the high-symmetry sites

In [ ]:
maps.sites()

In [ ]:
maps.cluster_sizes

In [ ]:
pos = maps.data.loc[maps.centroids,['x','y','z']].values
for i in range(pos.shape[0]):
    print("O "+" ".join(str(x) for x in pos[i,:]))

Visualize the results

In [ ]:
def scatter(self, feature = None, index = None, axes=['x','y'], region=0, categorical=False, centroids=False, splitby=None, splitby_index=None, set_aspect='on', **kwargs) -> None:
    """
    
    """
    # Filter data by region
    filterdata = self.data[self.contactspace.data['region'] == region]
    # Check that contact space maps have been generated
    if self.data is None:
        raise RuntimeError("No contact space data available.")
    # Check if feature or index is provided and if it is valid
    if feature is not None:
        if feature not in self.data.columns:
            raise ValueError(f"Feature {feature} not found in maps data.")
    elif index is not None:
        if index >= len(self.features) or index < 0:
            raise ValueError(f"Index {index} out of bounds.")
        feature = self.features[index]
        print(f"Plotting feature {self.features[index]}")
    else:
        f = None # No feature provided
    for axis in axes:
        if axis not in self.data.columns:
            raise ValueError(f"Axis {axis} not found in maps data.")
    if set_aspect not in ['on','off','equal','scaled']:
        raise ValueError(f"set_aspect must be one of ['on','off','equal','scaled']")
    # Select the axes for the plot
    x1 = filterdata[axes[0]].values.astype(np.float64)
    x2 = filterdata[axes[1]].values.astype(np.float64)
    # Select the axis to plot
    f = filterdata[feature].values.astype(np.float64)
    # Select the axis for splitting
    if splitby is not None:
        if splitby not in self.data.columns:
            raise ValueError(f"Split axis {splitby[0]} not found in maps data.")
        s = filterdata[splitby].values
        if splitby_index is not None:
            if splitby_index >= len(np.unique(s)) or splitby_index < 0:
                raise ValueError(f"Split index {splitby_index} out of bounds.")
            nsplit = 1
            fig, axs = plt.subplots(nsplit, 1, figsize=(8, 4*nsplit))
            axslist = [axs]
        else:
            nsplit = np.unique(s).size
            fig, axs = plt.subplots(nsplit, 1, figsize=(8, 4*nsplit))
            axslist = axs.flat
    else:
        nsplit = 1
        fig, axs = plt.subplots(nsplit, 1, figsize=(8, 4*nsplit))
        axslist = [axs]
    if categorical : 
        nf = np.unique(f).size
        colors = [ c for c in map(plt.cm.tab20, range(nf))]
    else:
        fmin = np.min(f)
        fmax = np.max(f)
    fig.subplots_adjust(hspace=0.3)
    # Plot the data    
    # Generate 2D plots for each unique value of the split variable
    for i,ax in enumerate(axslist):
        if splitby is not None:
            if splitby_index is not None:
                sval = np.unique(s)[splitby_index]
            else:
                sval = np.unique(s)[i]
            ax.set_title(f"Map of {feature} for {splitby} = {sval:6.2f}")
            mask = filterdata[splitby]==sval
            x1m = x1[mask]
            x2m = x2[mask]
            fm = f[mask]
        else:
            ax.set_title(f"Map of {feature}")
            x1m = x1
            x2m = x2
            fm = f
        ax.set_xlabel(f"{axes[0]}")
        ax.set_ylabel(f"{axes[1]}")
        if not categorical:
            if f is None:
                scatter = ax.scatter(x1m,x2m,**kwargs)
            else:
                scatter = ax.scatter(x1m,x2m,c=fm,vmin=fmin,vmax=fmax,**kwargs)
        else:
            for i,fvalue in enumerate(np.unique(f)):
                scatter = ax.scatter(x1m[fm==fvalue],x2m[fm==fvalue],color=colors[i],label=f'{feature} = {i:2}',**kwargs)
        if centroids: 
            if self.centroids is None:
                raise RuntimeError("No centroids available.")
            pos = self.data.loc[self.centroids,axes].values
            if splitby is not None:
                posmask = self.data.loc[self.centroids,splitby].values == sval
            else:
                posmask = np.ones(pos.shape[0],dtype=bool)
            ax.scatter(pos[posmask,0],pos[posmask,1],c='black',marker='x',label='Centroids',**kwargs)
        if categorical:
            leg = ax.legend(bbox_to_anchor=(1.01, 0.5), loc="center left")
            for lh in leg.legend_handles: 
                lh.set_alpha(1)
        ax.axis(set_aspect)
    if not categorical and f is not None:
        if nsplit > 1 :
            colorbar = fig.colorbar( scatter, ax=axs.ravel().tolist())
        else:
            colorbar = fig.colorbar( scatter, ax=ax)
        colorbar.solids.set_alpha(1.0)    
    return fig, axs

In [ ]:
fig, ax = scatter(maps,feature='Cluster', categorical=True, alpha=0.8, s=20, splitby='z', splitby_index=2, set_aspect='scaled', centroids=True)
ax.set_xlabel('x (Å)')
ax.set_ylabel('y (Å)')
ax.set_title('Clusters')
if publication:
    ax.set_title('')
    ax.get_legend().remove()
plt.show()

In [ ]:
fig, ax = scatter(maps,feature='Cluster', categorical=True, alpha=0.1, s=10, set_aspect='scaled', centroids=False)
ax.set_xlabel('x (Å)')
ax.set_ylabel('y (Å)')
ax.set_title('Clusters')
if publication:
    ax.set_title('')
    ax.get_legend().remove()
plt.show()

In [ ]:
axes = ['pca0','pca1']
fig, ax = maps.scatter(feature='Cluster', categorical=True, axes=axes, alpha=0.2, s=70, edgecolors='black', set_aspect='on')
G = nx.from_numpy_array(maps.cluster_edges,create_using=nx.DiGraph,parallel_edges=False)
pos = maps.data.loc[maps.centroids,axes].values
weights = [ d['weight']/4000 for (u, v, d) in G.edges(data=True)]
nx.draw(G, pos, node_size=maps.cluster_sizes/50, width=weights, ax=ax, alpha=0.8, edgecolors='black')
limits=ax.axis('on') # turns on axis
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_title('Clusters')
if publication:
    ax.set_title('')
    ax.get_legend().remove()
plt.show()